# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_csv = '../output_data/city_data.csv'
city_df = pd.read_csv(city_data_csv)
city_df

,City,Country,Cloud,Date,Humid,Lat,Lng,Max_temp,Temp,Wind
0,Cherdyn',RU,94,1648093198,89,60.4030,56.4787,20.26,20.26,2.39
1,Port Elizabeth,ZA,0,1648093198,86,-33.9180,25.5701,69.31,68.81,18.41
2,Tuktoyaktuk,CA,20,1648093198,76,69.4541,-133.0374,-14.80,-14.80,17.27
3,Egvekinot,RU,15,1648093198,82,66.3167,-179.1667,13.50,13.50,1.41
4,Katsuura,JP,72,1648093076,64,35.1333,140.3000,54.10,54.10,5.17
...,...,...,...,...,...,...,...,...,...,...
581,West Des Moines,US,100,1648093441,88,41.5720,-93.7453,36.28,35.80,14.97
582,Boaco,NI,90,1648093441,82,12.4722,-85.6586,71.24,71.24,5.93
583,Hīt,IQ,2,1648093442,71,33.6416,42.8251,51.49,51.49,7.74
584,Mossamedes,AO,90,1648093442,85,-15.1961,12.1522,73.80,73.80,4.92


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#  Configure gmaps.
gmaps.configure(api_key = g_key)

In [4]:
city_df = city_df.dropna()

locations = city_df[['Lat', 'Lng']]
humidity = city_df['Humid']

fig = gmaps.figure(center=[0,0], zoom_level =2)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    - A max temperature lower than 80 degrees but higher than 70.
    - Wind speed less than 10 mph.
    - Zero cloudiness.
* Drop any rows will null values.

In [5]:
weather_df = city_df.loc[(city_df['Max_temp']<80)
                              & (city_df['Max_temp']>70)
                              & (city_df['Wind']<70)
                              & (city_df['Cloud']==0),:].dropna()
weather_df

,City,Country,Cloud,Date,Humid,Lat,Lng,Max_temp,Temp,Wind
12,Busselton,AU,0,1648093201,35,-33.6500,115.3333,77.02,77.02,9.13
48,Beira,MZ,0,1648093219,100,-19.8436,34.8389,71.69,71.69,4.61
81,Beloha,MG,0,1648093231,68,-25.1667,45.0500,73.58,73.58,7.18
83,Port Augusta,AU,0,1648093232,34,-32.5000,137.7667,77.50,77.50,12.91
138,Saint-Louis,SN,0,1648093265,24,16.3333,-15.0000,74.84,74.84,12.30
204,Acapulco de Juárez,MX,0,1648093291,88,16.8634,-99.8901,75.02,75.02,6.91
291,Abu Dhabi,AE,0,1648093055,58,24.4667,54.3667,73.96,71.94,3.44
342,East London,ZA,0,1648093127,83,-33.0153,27.9116,70.74,70.74,9.22
350,Taoudenni,ML,0,1648093349,14,22.6783,-3.9836,73.44,73.44,8.93
450,Laguna,US,0,1648093387,62,38.4210,-121.4238,71.89,67.41,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_df[['City', 'Country', 'Lat', 'Lng']].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
12,Busselton,AU,-33.6500,115.3333,
48,Beira,MZ,-19.8436,34.8389,
81,Beloha,MG,-25.1667,45.0500,
83,Port Augusta,AU,-32.5000,137.7667,
138,Saint-Louis,SN,16.3333,-15.0000,
204,Acapulco de Juárez,MX,16.8634,-99.8901,
291,Abu Dhabi,AE,24.4667,54.3667,
342,East London,ZA,-33.0153,27.9116,
350,Taoudenni,ML,22.6783,-3.9836,
450,Laguna,US,38.4210,-121.4238,


In [7]:
params = {'radius':5000,
        'types':'lodging',          
        'key':g_key}

for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    
    params['lcoation']= f'{lat},{lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Retrieving results for city: {row['City']}")
    hotel_name = requests.get(base_url, params = params).json()
    
    hotel_name = hotel_name['results']
    
#     try: 
#     hotel_df.loc[index, "Hotel Name"] = hotel_name[0]['name']
#         print(f"Closest hotel is {hotel_name[0]['name']}.")
#         hotel_df.loc[index, 'Hotel Name'] = hotel_name[0]['name']

#     except (KeyError, IndexError):
#         print('Missing field/result... skipping.')
#     print('------------------')

hotel_df

Retrieving results for city: Busselton
Retrieving results for city: Beira
Retrieving results for city: Beloha
Retrieving results for city: Port Augusta
Retrieving results for city: Saint-Louis
Retrieving results for city: Acapulco de Juárez
Retrieving results for city: Abu Dhabi
Retrieving results for city: East London
Retrieving results for city: Taoudenni
Retrieving results for city: Laguna
Retrieving results for city: Morondava
Retrieving results for city: Rio de Janeiro
Retrieving results for city: Hervey Bay
Retrieving results for city: Manama
Retrieving results for city: Jalalabad


,City,Country,Lat,Lng,Hotel Name
12,Busselton,AU,-33.6500,115.3333,
48,Beira,MZ,-19.8436,34.8389,
81,Beloha,MG,-25.1667,45.0500,
83,Port Augusta,AU,-32.5000,137.7667,
138,Saint-Louis,SN,16.3333,-15.0000,
204,Acapulco de Juárez,MX,16.8634,-99.8901,
291,Abu Dhabi,AE,24.4667,54.3667,
342,East London,ZA,-33.0153,27.9116,
350,Taoudenni,ML,22.6783,-3.9836,
450,Laguna,US,38.4210,-121.4238,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig.
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))